# Projeto Spotify - SVM (Support Vector Machine)

Projeto de Machine Learning no curso Data Science do Zero da Stack Academy utilizando o método SVM desenvolvido. O objetivo é a modelar dados do Spotify para a criar um classificador que identifica possíveis músicas que um usuário poderia gostar.

# Features do conjunto de dados
<a href="https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features">Informações retiradas do site oficial do Spotify.</a>

- **acousticness (*float*):** Varia de 0.0 a 1.0 e indica o quão acústica é a faixa.
- **analysis_url (*string*):** Uma URL de acesso a uma análise completa da faixa. t
- **danceability (*float*):** Varia de 0.0 a 0.1 e indica o quão dancante é a faixa de acordo com uma combinação de diversos elementos musicais.
- **duration_ms (*int*):** A duração da faixa em milissegundos.
- **energy (*float*):** Varia de 0.0 to 1.0 e representa uma medida perceptível de intensidade e atividade.
- **id (*int*):** A ID da faixa no Spotify.
- **instrumentalness (*float*):** Indica se a faixa é instrumental. Valores acima de 0.5 sugerem que a faixa não contém vocais, sendo um indicador mais confiável quanto mais próximo de 1.0.
- **key (*int*):** Indica o tom da música. Se nenhum tom for identificado, a feature assume o valor 0.
- **liveness (*float*):** Indica a presença de platéia na gravaçao. Um valor superior a 0.8 sugere fortemente que a faixa foi gravada ao vivo.
- **loudness (*float*):** A intensidade sonora em decibéis (dB).
- **mode (*int*):** Indica a escala da faixa. Escala maior é representada por 1 e escala menor é representada por 0.
- **speechiness (*float*):** Indica a presença de palavras faladas na faixa. Valores maiores que 0.66 sugerem que provavelmente a faixa é inteiramente falada, enquanto valores menores que 0.33 costumam representar músicas ou outras faixas não-faladas. 
- **tempo (*float*):** O andamento da faixa em batidas por minuto (BPM).
- **time_signature (*int*):** Representa a métrica da música e varia de 3 a 7, em referência às métricas de 3/4 a 7/4.
- **track_href (*string*):** Um link que contém detalhes da faixa. 
- **type (*string*):** O tipo de objeto ("audio_features")
- **uri (*string*):** A URI da faixa no Spotify.
- **valence (*float*):** Varia de 0.0 a 1.0 e indica a positividade da faixa. Valores acima de 0.5 sugerem um som mais alege e eufórico, enquanto valores mais baixos costumam soar mais tristes.
- **target (*int*):** Indica se o usuário gosta (1) ou não (0) da faixa.
- **song_title (*string*):** Título da faixa.
- **artist (*string*):** Artista da faixa.

# Etapas do projeto

1. Leitura e análise preliminar dos dados
2. Pré-processamento de dados
3. Modelagem 

# 1. Leitura e análise preliminar dos dados

### **Bibliotecas utilizadas**

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn import metrics, svm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

### **Leitura dos dados**

In [ ]:
dados = pd.read_csv('spotify.csv')
dados.sample(10)

### **Análise preliminar dos dados e geração de relatório em html com informações estatíticas**

In [ ]:
dados.info() 

In [ ]:
dados.describe() 

In [ ]:
from pandas_profiling import ProfileReport

relatorio = ProfileReport(dados, title='Relatório Spotify') 
relatorio.to_file('relatorio_spotify.html') 

- Não há valores faltantes nem dados duplicados!
- Pela análise do calor de mapa das correlações de Pearson presentes no relatório, percebe-se uma correlação negativa moderada entre loudness e acousticness, uma correlação negativa forte entre energy e acousticness, e uma correlação positiva forte entre loudness e energy.

### **Visualização das preferências do usuário.**

- Features analisadas: 'acousticness','danceability','energy','instrumentalness','loudness','speechiness','tempo','valence' e 'target'.
- Pontos verdes: Usuário não gosta da faixa
- Pontos laranjas: Usuário gosta da faixa

In [ ]:
legenda = pd.cut(dados['target'],2,labels=['Não gosta','Gosta'])
dados['Legenda'] = legenda

%matplotlib notebook
features = ['acousticness','danceability','energy','instrumentalness','loudness','speechiness','tempo','valence','Legenda']
f = sns.pairplot(dados[features], hue='Legenda', palette = 'Set2')
plt.savefig('pairplot.pdf')
f.savefig('pairplot.pdf') 

**Inicialmente, pela análise das interações das features não é possível identificar um padrão claro entre as músicas aprovadas e as não apovadas pelo usuário.**

# 2. Pré-processamento de dados

### **Separação da classe de interesse: target.**

In [ ]:
classe = dados['target']
dados.drop('target', axis=1, inplace=True)
dados.head()

### **Remoção de features que não tem contribuição no modelo.**

In [ ]:
def remove_features(lista_features):
    dados.drop(lista_features, axis=1, inplace=True)
    return

remove_features(['id','song_title','Legenda'])
dados.head()

### **Transformação de variáveis categóricas.**
- O método SVM não funciona para dados categóricos, como no caso da feature artist.
- Por isso, convém usar técnicas para codificar valores categoricos em numéricos.
- O Pandas Get-dummies é uma técnica que converte uma lista categórica de *n* itens únicos em uma matriz de presença de *n* colunas com valores binários que indicam a presença (1) ou não (0) de cada variável em cada linha, e concatena essa matriz ao conjunto de dados. Sua desvantagem, no entanto, é **alta dimensionalidade** do novo conjunto de dados que pode geras um dataframe muito esparso quando as features apresentam muitos valores distintos.

In [ ]:
dados_gd = pd.get_dummies(dados, columns=['artist'], prefix=['artist'])
dados_gd

- LabelEnconder é uma técnica que converte uma lista categórica como _(‘branco’,’preto’,’amarelo’,’vermelho’,'branco')_ em outra númerica como __(1,2,3,4,1)__, superando o problema de alta dimensionalidade gerada por técnicas como o Pandas GetDummies. No entanto, este método pode causar problemas de **ordenação**, atribuindo pesos aos dados categóricos.

In [ ]:
from sklearn.preprocessing import LabelEncoder

codificador = LabelEncoder() #istancia o método 
inteiros = codificador.fit_transform(dados['artist']) #codifica os valores da variável categórica
dados['artist_int'] = inteiros #adiciona uma coluna com os valores inteiros
remove_features(['artist']) #remove a coluna categorica 
dados.head()

# 3. Modelagem

### Aplicação de pipelines com diferentes kernels para padronizar os dados de maneira automatizada

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

sem_pip = svm.SVC().fit(dados,classe)

pip_1 = Pipeline([
                   ('scaler', StandardScaler()),  
                   ('classificador', svm.SVC())   
                 ])

pip_2 = Pipeline([
                   ('min_max_scaler', MinMaxScaler()),  
                   ('classificador', svm.SVC())   
                 ])

pip_3 = Pipeline([
                   ('scaler', StandardScaler()), 
                   ('classificador', svm.SVC(kernel='poly'))  
                 ])

pip_4 = Pipeline([
                   ('scaler', StandardScaler()), 
                   ('classificador', svm.SVC(kernel='linear'))  
                 ])

### Cálculo da acurácia e teste das técnicas de transformação dos dados categóricos

In [ ]:
def acuracia(clf, x, y):
    resultados = cross_val_predict(clf, x, y, cv=10)
    ac = metrics.accuracy_score(y,resultados)
    return ac

In [ ]:
print('Acurácia com GetDummies\n')
print('Sem pipeline:',acuracia(sem_pip, dados_gd, classe))
print('Pipeline 1:  ',acuracia(pip_1,dados_gd,classe))
print('Pipeline 2:  ',acuracia(pip_2,dados_gd,classe))
print('Pipeline 3:  ',acuracia(pip_3,dados_gd,classe))
print('Pipeline 4:  ',acuracia(pip_4,dados_gd,classe))

In [ ]:
print('Acurácia com LabelEncoder\n')
print('Sem pipeline:',acuracia(sem_pip, dados, classe))
print('Pipeline 1:  ',acuracia(pip_1,dados,classe))
print('Pipeline 2:  ',acuracia(pip_2,dados,classe))
print('Pipeline 3:  ',acuracia(pip_3,dados,classe))
print('Pipeline 4:  ',acuracia(pip_4,dados,classe))

**Apesar do problema de ordenação, o conjunto de dados tratado com LabelEncoder apresentou maior acurácia que o conjunto tratado com GetDummies para todos os pipelines, com exceção do pip_2. De fato, a dimensionalidade do dataset aumentou excessivamente com o uso do GetDummies, prejudicando a performance do modelo.** 

### Otimização dos parâmetros do modelo usando pip_1, que apresentou maior acurácia

In [ ]:
from sklearn.model_selection import GridSearchCV

lista_C = [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]

lista_gama = [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2]

parametros_grid = dict(classificador__C = lista_C, classificador__gamma = lista_gama)
grid = GridSearchCV(pip_1, parametros_grid, cv=10, scoring='accuracy')
grid.fit(dados,classe)

In [ ]:
C = grid.best_params_.get('classificador__C')
gama = grid.best_params_.get('classificador__gamma')

print('Parâmetro C otimizado =', C)
print('Parâmetro gama otimizado =', gama)